In [1]:
# Dependencies

import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pylab as plt
%matplotlib inline

In [2]:
# Import modules for feature engineering and modelling
from sklearn.model_selection import train_test_split  # not used here, split based on provider
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# for integer encoding using sklearn
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

from sklearn import metrics 
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import plot_roc_curve  


from sklearn.metrics import precision_recall_curve
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_curve.html

from sklearn.metrics import auc,plot_precision_recall_curve
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.auc.html

from sklearn.metrics import roc_curve,auc,RocCurveDisplay

# Set seed for reproducibility
seed1, seed2, seed3=101,67, 42

## Load and Merge Data

In [2]:
# load the data from csv to pandas dataframe
X_test_aggregated_raw = pd.read_csv("output/agg_train_test_split/X_test_aggregated.csv")
X_train_aggregated_raw = pd.read_csv("output/agg_train_test_split/X_train_aggregated.csv")
y_test_aggregated_raw = pd.read_csv("output/agg_train_test_split/y_test_agg.csv")
y_train_aggregated_raw = pd.read_csv("output/agg_train_test_split/y_train_agg.csv")

In [3]:
# Assign inpatient data as 1, outpatient data as 0
Healthcare_Inpatient_raw['In_Out']=1
Healthcare_Outpatient_raw['In_Out']=0

# union/concat the inpatient and outpatient data
concat_df=pd.concat([Healthcare_Inpatient_raw,Healthcare_Outpatient_raw],axis=0)

# left join tables
merge_bene_df=concat_df.merge(Healthcare_Beneficiary_raw, on='BeneID', how='left')
merge_provider_df=merge_bene_df.merge(Healthcare_Provider_raw, on = 'Provider', how ='left')
merge_provider_df.drop_duplicates(inplace=True)

# remove columns with all null values
merge_provider_df.dropna(axis=1, how='all',inplace=True)

# check the final merged dataframe
merge_provider_df

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,...,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,PotentialFraud
0,BENE11001,CLM46614,2009-04-12,2009-04-18,PRV55912,26000,PHY390922,NaN,NaN,2009-04-12,...,1,1,2,1,1,36000,3204,60,70,Yes
1,BENE11001,CLM66048,2009-08-31,2009-09-02,PRV55907,5000,PHY318495,PHY318495,NaN,2009-08-31,...,1,1,2,1,1,36000,3204,60,70,No
2,BENE11001,CLM68358,2009-09-17,2009-09-20,PRV56046,5000,PHY372395,NaN,PHY324689,2009-09-17,...,1,1,2,1,1,36000,3204,60,70,No
3,BENE11011,CLM38412,2009-02-14,2009-02-22,PRV52405,5000,PHY369659,PHY392961,PHY349768,2009-02-14,...,1,2,2,1,1,5000,1068,250,320,No
4,BENE11014,CLM63689,2009-08-13,2009-08-30,PRV56614,10000,PHY379376,PHY398258,NaN,2009-08-13,...,2,1,2,2,2,21260,2136,120,100,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558206,BENE159198,CLM510792,2009-08-06,2009-08-06,PRV53699,800,PHY364188,PHY364188,PHY385752,NaN,...,1,2,2,1,2,0,0,5470,1870,No
558207,BENE159198,CLM551294,2009-08-29,2009-08-29,PRV53702,400,PHY423019,PHY332284,NaN,NaN,...,1,2,2,1,2,0,0,5470,1870,No
558208,BENE159198,CLM596444,2009-09-24,2009-09-24,PRV53676,60,PHY361063,NaN,NaN,NaN,...,1,2,2,1,2,0,0,5470,1870,No
558209,BENE159198,CLM636992,2009-10-18,2009-10-18,PRV53689,70,PHY403198,NaN,PHY419379,NaN,...,1,2,2,1,2,0,0,5470,1870,No


## Basic Transformation

In [5]:
# seperate columns into target, num, datetime, categorical
target=['PotentialFraud']
colnames=merge_provider_df.columns.to_list()
num_col_list=(merge_provider_df.describe()).columns.tolist()
datetime_col_list=['ClaimStartDt','ClaimEndDt','DischargeDt','DOB','DOD']
cat_col_list=list(set(colnames)-set(num_col_list)-set(target)-set(datetime_col_list))

In [6]:
# get a copy of the merged data, prepare change all to numerical
df_num=merge_provider_df.copy(deep=True)

# change date column datatype to datetime

for datetime_col_name in datetime_col_list:
    df_num[datetime_col_name]=pd.to_datetime(df_num[datetime_col_name])

    
# label encode all object columns

le=LabelEncoder()

for cat_col_name in cat_col_list:
    df_num[cat_col_name]=le.fit_transform(df_num[cat_col_name])

    
# transform target to binary code

df_num['PotentialFraud']=df_num['PotentialFraud'].map(lambda x:1 if (x=='Yes' or x==1) else 0)

df_num.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,...,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,PotentialFraud
0,9381,299052,2009-04-12,2009-04-18,3928,26000,53275,35315,46457,134,...,1,1,2,1,1,36000,3204,60,70,1
1,9381,471344,2009-08-31,2009-09-02,3923,5000,4951,2132,46457,275,...,1,1,2,1,1,36000,3204,60,70,0
2,9381,491764,2009-09-17,2009-09-20,4034,5000,40843,35315,5201,292,...,1,1,2,1,1,36000,3204,60,70,0
3,9489,226452,2009-02-14,2009-02-22,1125,5000,39020,23575,14717,77,...,1,2,2,1,1,5000,1068,250,320,0
4,9518,450337,2009-08-13,2009-08-30,4490,10000,45475,25118,46457,257,...,2,1,2,2,2,21260,2136,120,100,0


## Transform Datatime columns + Add Code Count

In [7]:
# calculte claimPeriod and HospitalizedDays
# df_num = merge_provider_df.copy(deep=True)
df_num['ClaimPeriod'] = (pd.to_datetime(df_num.ClaimEndDt) - pd.to_datetime(df_num.ClaimStartDt)).dt.days + 1
df_num['HospitalizedDays'] = (pd.to_datetime(df_num.DischargeDt) - pd.to_datetime(df_num.AdmissionDt)).dt.days + 1
# make hospitallized day 0 for outpatient
df_num['HospitalizedDays'] = df_num['HospitalizedDays'].apply(lambda x : 0 if pd.isna(x) else x )

In [8]:
#calculate age and determine if a person is deceased. If the person is still alive, 
#use '2009-12-01' as the time we calculate age

from datetime import date, datetime
def calulateAge(dob, dod, calulationDate):
    born = dob
    if not pd.isna(dod):
        calulationDate = dod
    else:
        calulationDate = datetime.strptime(calulationDate, "%Y-%m-%d").date()
    return calulationDate.year - born.year - ((calulationDate.month, calulationDate.day) < (born.month, born.day))    


df_num['Deceased'] = df_num['DOD'].apply(lambda x : 0 if pd.isna(x) else 1) 
df_num['Age'] = df_num.apply(lambda x: calulateAge(dob = x['DOB'], dod = x['DOD'], calulationDate = '2009-12-01'), axis = 1)

In [9]:
'''columns: list of columns ['ClmDiagnosisCode_1', 'ClmDiagnosisCode_2']'''
def countCodeNumbers(df, columns):
    df_codes = df_num.loc[:, columns]
    codecount = df_codes.notnull().sum(axis=1)
    return codecount

diagnosis_code_columns = [x for x in list(df_num.columns) if 'ClmDiag' in x]
procedure_code_columns = [x for x in list(df_num.columns) if 'ClmProc' in x]

df_num['DiagnosisCodeCount'] = countCodeNumbers(df_num, diagnosis_code_columns)
df_num['ProcedureCodeCount'] = countCodeNumbers(df_num, procedure_code_columns)

In [10]:
df_num['DiagnosisGroupCode'] = df_num['DiagnosisGroupCode'].apply(lambda x: -999 if pd.isna(x) else x)